In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Week5_Structured_Streaming").getOrCreate()

In [2]:
import os

input_dir = '/home/jovyan/work/data/streaming_inbox/'
os.makedirs(input_dir, exist_ok=True)
print("Streaming input folder ready:", input_dir)

Streaming input folder ready: /home/jovyan/work/data/streaming_inbox/


In [3]:
# NOTE: Files MUST have the same schema
stream_df = spark.readStream \
    .option("header", True) \
    .schema(spark.read.csv("/home/jovyan/work/data/sales.csv", header=True, inferSchema=True).schema) \
    .csv(input_dir)

stream_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: double (nullable = true)



In [4]:
agg_df = stream_df.groupBy("product").sum("quantity")

In [5]:
query = agg_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

# This runs the stream job—for a notebook, try either:
# query.awaitTermination(60)   # Run for 60 seconds then stop
# OR
# query.processAllAvailable()  # For batch testing (PySpark 3.2+)